In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold,TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder
from dateutil.relativedelta import relativedelta
from datetime import timedelta
import tqdm

import random
from collections import Counter,defaultdict

from copy import deepcopy
sns.set_theme(style="ticks")
# データを保存しているフォルダのディレクトリ指定
DATA_DIR = './data/'
OUTPUT_DIR = "output"

# データ読み込み

submission = pd.read_csv(os.path.join(DATA_DIR, 'sample_submit.csv'), header = None)

status_df = pd.read_csv(os.path.join(DATA_DIR, 'status.csv'))
tmp = status_df['year'].astype(str).str.cat(status_df['month'].astype(str), sep='/')
tmp = tmp.str.cat(status_df['day'].astype(str), sep='/')
#tmp = tmp.str.cat(status_df['hour'].astype(str), sep='/')
#status_df["date"] = pd.to_datetime(tmp,format='%Y/%m/%d/%H')
status_df["date"] = pd.to_datetime(tmp,format='%Y/%m/%d')


trip_df = pd.read_csv(os.path.join(DATA_DIR, 'trip.csv'))
trip_df["start_date"] = pd.to_datetime(trip_df.start_date)
trip_df["end_date"] = pd.to_datetime(trip_df.end_date)

weather_df = pd.read_csv(os.path.join(DATA_DIR, 'weather.csv'))
weather_df['date'] = pd.to_datetime(weather_df['date'])

station_df = pd.read_csv(os.path.join(DATA_DIR, 'station.csv'))

In [3]:
trip_df

,trip_id,duration,start_date,start_station_id,end_date,end_station_id,bike_id,subscription_type
0,0,63,2013-08-29 14:13:00,54,2013-08-29 14:14:00,54,0,Subscriber
1,1,70,2013-08-29 14:42:00,8,2013-08-29 14:43:00,8,1,Subscriber
2,2,71,2013-08-29 10:16:00,20,2013-08-29 10:17:00,20,2,Subscriber
3,3,77,2013-08-29 11:29:00,8,2013-08-29 11:30:00,8,3,Subscriber
4,4,83,2013-08-29 12:02:00,54,2013-08-29 12:04:00,55,4,Subscriber
...,...,...,...,...,...,...,...,...
669954,669954,619,2014-09-01 04:21:00,41,2014-09-01 04:32:00,53,593,Subscriber
669955,669955,6712,2014-09-01 03:16:00,39,2014-09-01 05:08:00,58,51,Customer
669956,669956,538,2014-09-01 00:05:00,54,2014-09-01 00:14:00,45,248,Customer
669957,669957,568,2014-09-01 00:05:00,54,2014-09-01 00:15:00,45,236,Customer


In [70]:
_tmp = status_df[status_df.date < "2014-09-01"]
all_dates = sorted(_tmp.date.unique())
all_dates = _tmp.date.unique()
all_station_ids = status_df.station_id.unique()

In [71]:
from tqdm.notebook import tqdm
pbar = tqdm(total=len(all_dates) * len(all_station_ids))

supply_df = pd.DataFrame()

dates = []
supply = []
station_ids = []
for idx, d in enumerate(all_dates):
    before = d
    after = np.timedelta64(1,'D') + d
    index = (before <= trip_df.start_date) & (trip_df.start_date < after)
    one_day_trip = trip_df[index]

    for j, station_id in enumerate(all_station_ids):
        pbar.update(1)

        out_bike_num = len(one_day_trip[(one_day_trip.start_station_id == station_id)])
        in_bike_num = len(one_day_trip[(one_day_trip.end_station_id == station_id)])

        y = status_df[(status_df.station_id == station_id)&(status_df.date == d)]
        start_bike = y.bikes_available.values[0]
        if np.isnan(start_bike):
            continue

        y2 = status_df[(status_df.station_id == station_id)&(status_df.date == after)]
        start_bike2 = y2.bikes_available.values[0]
        bike_diff = start_bike - out_bike_num + in_bike_num
        dates.append(d)
        supply.append(start_bike2 - bike_diff)
        station_ids.append(station_id)

supply_df["date"] = dates
supply_df["supply"] = supply
supply_df["station_id"] = station_ids
supply_df.head()

,date,supply,station_id
0,2013-09-01,1.0,0
1,2013-09-01,0.0,1
2,2013-09-01,0.0,2
3,2013-09-01,0.0,3
4,2013-09-01,0.0,4


In [ ]:
one_day_trip[one_day_trip.start_station_id == 0]

In [17]:
one_day_trip[one_day_trip.end_station_id == 0]

,trip_id,duration,start_date,start_station_id,end_date,end_station_id,bike_id,subscription_type
9369,9369,538,2013-09-11 07:47:00,5,2013-09-11 07:56:00,0,490,Subscriber
9412,9412,123,2013-09-11 08:21:00,0,2013-09-11 08:23:00,0,489,Subscriber
9702,9702,391,2013-09-11 13:19:00,4,2013-09-11 13:26:00,0,423,Subscriber
9875,9875,397,2013-09-11 16:44:00,4,2013-09-11 16:50:00,0,396,Subscriber
9889,9889,749,2013-09-11 16:59:00,9,2013-09-11 17:11:00,0,278,Customer
9900,9900,419,2013-09-11 17:06:00,3,2013-09-11 17:13:00,0,559,Subscriber
9944,9944,666,2013-09-11 17:33:00,11,2013-09-11 17:44:00,0,479,Subscriber
10036,10036,283,2013-09-11 18:37:00,2,2013-09-11 18:42:00,0,366,Customer
10044,10044,673,2013-09-11 18:43:00,11,2013-09-11 18:54:00,0,295,Subscriber
10149,10149,574,2013-09-11 20:56:00,9,2013-09-11 21:06:00,0,407,Customer


In [72]:
_ind = (status_df.station_id == 0) & (status_df.hour == 0)
status_df[_ind].head(5)

,id,year,month,day,hour,station_id,bikes_available,predict,date
0,0,2013,9,1,0,0,11.0,0,2013-09-01
24,24,2013,9,2,0,0,12.0,0,2013-09-02
48,48,2013,9,3,0,0,10.0,0,2013-09-03
72,72,2013,9,4,0,0,9.0,0,2013-09-04
96,96,2013,9,5,0,0,15.0,0,2013-09-05


In [79]:
supply_df = supply_df.sort_values([ "station_id","date"])
supply_df.to_csv("data/supply.csv")